### Nam Tu - 153076622 - Pyspark and MapReduce

### Task 1.1

### Task 1.2

1. The code generates numbers from 1 to (amount of numbers per partition) * no.partitions , the entire set is then partitioned into 10 parts. The 10 parts are then passed off onto a MapReduce process which map a truth value to each value (1 if a randomly generated point lies in a circle of radius 1, 0 otherwise). The total number of points is then calculated via aggregation in the reduce stage. From which the value of pi can be estimated  via the formula 4*(no. of points in the circle)/no. of points generated.

2. sc.parrallelize takes the entire dataset and turns it into an RDD with the number of partitions stated (10 in this case), which will enable the ability to do parrallel computations (hence the name). The RDD can then be passed off onto a MapReduce process, making the computation much faster.

3. 

4. Increasing the amount of data moves the results towards the true value of pi, decreasing does the opposite.

In [3]:
from random import random
from operator import add
import pyspark
sc = pyspark.SparkContext("local", "CalculatePi")
partitions = 10
n1 = 100000 * partitions
n2 = 50000 * partitions
n3 = 300000 * partitions
#function which evaluates if the random x,y point is inside the unit circle
def f(_):
 x = random() * 2 - 1
 y = random() * 2 - 1
 return 1 if x ** 2 + y ** 2 < 1 else 0
#parallelize = make a range of keys and partition keys to x partitions
#assing a value to each key with .map(f)
#and finally reduce the values by summation
rdd1 = sc.parallelize(range(1, n1 + 1), partitions)
count1 = rdd1.map(f).reduce(add)
print("Pi  (with 100.000 points) is roughly %f" % (4.0 * count1 / n1))

rdd2 = sc.parallelize(range(1, n2 + 1), partitions)
count2 = rdd2.map(f).reduce(add)
print("Pi (with 50.000 points) is roughly %f" % (4.0 * count2 / n2))

rdd3 = sc.parallelize(range(1, n3 + 1), partitions)
count3 = rdd3.map(f).reduce(add)
print("Pi (with 300.000 points) is roughly %f" % (4.0 * count3 / n3))
sc.stop()

Pi  (with 100.000 points) is roughly 3.133760
Pi (with 50.000 points) is roughly 3.127840
Pi (with 300.000 points) is roughly 3.142160


### Task 1.3

The text file is fed into the instance of Spark, essentially making the textfile a RDD consisting of strings, Pyspark will do the paritition itself(?). The RDD is fed through 2 mapping functions before reducing. First, each line is formatted: punctuations and tabs removed. Each line is then split into words. Second, for each partition, we count the amount of time each word appears.

Finally, for the reduction stage, merge the count of words from each partition by adding them together. The output is the product after the MapReduce, displaying the words and their count.

In [1]:
import pyspark
sc = pyspark.SparkContext("local", "words")
lines = sc.textFile("*.txt")
words = lines.map(lambda line:
line.replace(".","").replace(",","").replace("?","").replace("\t","").replace(":","")
.replace("!","").replace("-","").lower()).flatMap(lambda line: line.split(" "))
keyvalue = words.map(lambda word: (word, 1))
counts = keyvalue.reduceByKey(lambda a, b: a + b)
#retrieve at most 200 key value pairs from the rdd (it’s not wise to try to get everything)
for x in counts.take(10):
 print(x)
sc.stop()

('the', 14512)
('project', 87)
('gutenberg', 26)
('ebook', 12)
('of', 6707)
('moby', 81)
('dick;', 10)
('or', 787)
('whale', 846)
('', 4316)
